In [9]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing,metrics 
from IPython.core.display import HTML
pd.set_option("display.max_columns",75)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import linear_model,svm
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

In [10]:
data = pd.read_csv('/home/gozmit/Datathon/lending_club_loans.csv')

In [11]:
df = pd.DataFrame()
df = data

In [12]:
df.drop(['id','emp_title','url','desc','title','zip_code','addr_state','mths_since_last_delinq','pymnt_plan','mths_since_last_record','initial_list_status','purpose','verification_status','issue_d','delinq_2yrs','earliest_cr_line','inq_last_6mths','pub_rec','out_prncp','out_prncp_inv'],axis=1,inplace=True)

In [13]:
df.shape

(42542, 96)

In [14]:
df.drop(df.columns[53:116],axis=1,inplace=True)

In [15]:
df.shape

(42542, 53)

In [29]:
len(df.columns)
df.columns

Index(['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term',
       'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length',
       'home_ownership', 'annual_inc', 'loan_status', 'dti', 'fico_range_low',
       'fico_range_high', 'open_acc', 'revol_bal', 'revol_util', 'total_acc',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'last_fico_range_high', 'last_fico_range_low',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m'],
      dtype='object')

In [17]:
df.shape

(42542, 53)

In [20]:
df_with_loanstatus_sliced = df[(df['loan_status']=='Fully Paid') | (df['loan_status']=='Charged Off')]

In [21]:
df_bool = {'Fully Paid': 0, 'Charged Off': 1}

In [22]:
df_bool_target = df_with_loanstatus_sliced.replace({'loan_status': df_bool})

In [23]:
df_bool_target.loan_status.value_counts()

0    33586
1     5653
Name: loan_status, dtype: int64

In [44]:
#df = df_bool_target.dropna(how='any',axis=0)
df_bool_target.head(2)
df_bool_target.shape

(39239, 32)

In [38]:
df_bool_target.columns[33]
df_bool_target.drop(df_bool_target.columns[33:53],axis=1,inplace=True)

In [43]:
df_bool_target.dropna(thresh=10000,axis=1,inplace=True) #next_pymnt_d

In [47]:
df_bool_target.collection_recovery_fee.value_counts()

0.0000      35420
2.0000         12
1.2000         10
3.7100          9
0.8000          8
            ...  
662.3080        1
4.6093          1
101.6400        1
69.8600         1
0.2700          1
Name: collection_recovery_fee, Length: 2655, dtype: int64

In [50]:
df_bool_target.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,loan_status,fico_range_low,fico_range_high,open_acc,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low
0,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,0,735.0,739.0,3.0,13648.0,83.7%,9.0,5863.155187,5833.84,5000.00,863.16,171.62,Sep-2016,744.0,740.0
1,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,1,740.0,744.0,3.0,1687.0,9.4%,4.0,1008.710000,1008.71,456.46,435.17,119.66,Sep-2016,499.0,0.0
2,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,0,735.0,739.0,2.0,2956.0,98.5%,10.0,3005.666844,3005.67,2400.00,605.67,649.91,Sep-2016,719.0,715.0
3,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,0,690.0,694.0,10.0,5598.0,21%,37.0,12231.890000,12231.89,10000.00,2214.92,357.48,Apr-2016,604.0,600.0
5,1311441.0,5000.0,5000.0,5000.0,36 months,7.90%,156.46,A,3 years,RENT,36000.0,0,730.0,734.0,9.0,7963.0,28.3%,12.0,5632.210000,5632.21,5000.00,632.21,161.03,Jan-2016,679.0,675.0


In [48]:
df_bool_target.drop(['sub_grade','dti','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d'],axis=True,inplace=True)

In [49]:
df_bool_target.shape

(39239, 26)

In [51]:
features = ['funded_amnt','emp_length','annual_inc','home_ownership','grade',
            "last_pymnt_amnt", "mort_acc", "pub_rec", "int_rate", "open_acc","num_actv_rev_tl",
            "mo_sin_rcnt_rev_tl_op","mo_sin_old_rev_tl_op","bc_util","bc_open_to_buy",
            "avg_cur_bal","acc_open_past_24mths",'loan_status'] #'sub_grade' #selecting final features #'addr_state''tax_liens',
Final_data = df_bool_target[features] #19 features with target var

KeyError: "['bc_util', 'pub_rec', 'avg_cur_bal', 'num_actv_rev_tl', 'acc_open_past_24mths', 'bc_open_to_buy', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mort_acc'] not in index"